In [15]:
        from nbformat import NotebookNode
        from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
        from nbconvert.filters import comment_lines
        from nbconvert.preprocessors import Preprocessor
        from functools import partial, wraps
        from traitlets import Any, Unicode, Bool
        from inspect import getfullargspec
        __all__ = tuple()

In [16]:
        def preprocessor(function):
            """A decorator to create nbconvert preprocessors; no arguments for the preprocessors."""
            args = getfullargspec(function).args
            if not args[0] == 'self':
                raise ValueError('Preprocessor must define `self`.')
            return type(
                function.__name__, (Preprocessor,), 
                {'preprocess' + {4: '_cell', 3: ''}[len(args)]: function,})

In [10]:
        @preprocessor
        def JoinSource(self, cell, resources, index):
            if not isinstance(cell['source'], str):
                cell['source'] = '\n'.join(cell['source'])
            return cell, resources

In [9]:
        @preprocessor
        def SplitSource(self, cell, resources, index):
            if isinstance(cell['source'], str):
                cell['source'] = cell['source'].splitlines()
            return cell, resources